In [1]:
import pandas
import numpy as np
import sklearn
import warnings
warnings.filterwarnings('ignore')

# Maciej Sączewski

## Systemy Inteligentne w Biznesie

### Przygotowanie danych - czyszczenie i przeżuwanie (przemielanie) danych

#### Wczytanie danych

In [2]:
plik_z_danymi = "./TitanicMess.tsv"

data = pandas.read_table(plik_z_danymi)
data = data.set_index(data.PassengerId)
del data["PassengerId"]

Podsumowanie wczytanych danych

Wypisanie danych

In [3]:
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic
...,...,...,...,...,...,...,...,...,...,...,...,...
888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30,B42,S,Titanic
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,"23,45",NaN,S,Titanic
890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30,C148,C,Titanic


Typ danych poszczególnych atrybutów

W przypadku danych jakościowych autmatycznie zostają uznane za typ 'object', tak samo w przypadku danych ilościowych, w których wystepują brakujące wartości. Dopiero po uzupełnieniu lub usnięciu mozna zmienic tym atrybutu.

In [4]:
pandas.DataFrame(data.dtypes)


,0
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,object
SibSp,int64
Parch,int64
Ticket,object
Fare,object
Cabin,object


Statystyka podsumowująca dla danych ilościowych

In [5]:
data.describe()

,Survived,Pclass,SibSp,Parch
count,892.000000,892.000000,892.000000,892.000000
mean,0.386771,2.307175,0.525785,0.371076
std,0.487284,0.836750,1.102190,0.768468
min,0.000000,1.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,0.000000
50%,0.000000,3.000000,0.000000,0.000000
75%,1.000000,3.000000,1.000000,0.000000
max,1.000000,3.000000,8.000000,5.000000


Dla danych w których występują braki lub błedne dane, może się okazać że został dobrany zły typ danych. Po naprawie błednych danych oraz wypełnieniu braków, typy danych zostaną wypisane jeszcze raz i w razie potrzeby przypisanę ręcznie.

### Zduplikowane wiersze

Ilośc wierszy zduplikowanych

In [6]:
len(data) - len(data.drop_duplicates())

3

Usunięcie wierszy zduplikowanych

In [7]:
data = data.drop_duplicates()

#### Błędne dane



Atrybut Age: wiersze zawierające liczbe całkowitą.

Liczba zmienno przecinkowa zostaje zamieniona na liczbe naturalną.

In [8]:
data["Age"] = data["Age"].astype("string")

for index, row in data.iterrows():
    if pandas.isna(row["Age"]):
        continue
    elif "," in row["Age"]:
        data.at[index, 'Age'] = str(int(float(row["Age"].replace(',','.'))))
    elif "." in row["Age"]:        
        data.at[index, 'Age'] = str(int(float(row["Age"])))
    

Atrybut Sex: literówki

Zastąpienie błednych zapisanych płci prawidłowymi

In [9]:
data["Sex"].unique()

array(['male', 'female', 'malef', 'mal', 'fem', 'femmale'], dtype=object)

In [10]:
for index, row in data.iterrows():
    if pandas.isna(row["Sex"]):
        continue
    elif row["Sex"] in ["malef", "mal"]:
        data.at[index, 'Sex'] = 'male'
    elif row["Sex"] in ["fem", "femmale"]:       
        data.at[index, 'Sex'] = 'female'

#### Braki danych w poszczególnych atrybutach

In [12]:
braki = pandas.DataFrame(data.isna().sum())
braki.columns = ["Ilość braków"]

procenty = [x / len(data) for x in data.isna().sum()]
procnety = np.array(procenty)
procenty = np.around(procenty, 3)

braki["Procent braków"] = procenty * 100

braki

,Ilość braków,Procent braków
Survived,0,0.0
Pclass,0,0.0
Name,0,0.0
Sex,0,0.0
Age,173,19.5
SibSp,0,0.0
Parch,0,0.0
Ticket,0,0.0
Fare,0,0.0
Cabin,685,77.1


W przypadku atrybutów których braki są większe niż 60%, atrybut jest wyłączany z dalszego przetwarzania oraz analizy. W przypadku tego przygotowania danych atrybut zostaje usuniety ze zbioru danych.

In [13]:
for index, row in braki.iterrows():
    if row["Procent braków"] > 60.0:
        del data[index]

braki = braki[braki['Procent braków'] < 60]

W przypadku pozostały atrybutów z brakującymi danymi zostaną one zastąpione wartościami średnimi w zaleznosci od atrybutu grupującego zdefinowanego przez użytkownika.

Uzytkownik uzpełnia słownik, w którym kluczem jest atrybut, który ma zostać uzupełniony, a jego wartościa atrybut grupujący. W przypadku gdy atrybut zalezny nie pozwala na obliczenie średniej zostaje zastosowana mediana.

In [14]:
do_uzupelnienia = {"Age": "Pclass", "Embarked": "Sex"}

W celu automatyzacji obliczania miediany oraz średniej korzystam z danych bez braków, tj. dane w których wszstkie wiersze z brakami są usunięte

In [15]:
dane_bez_brakow = data
dane_bez_brakow = dane_bez_brakow.dropna()
dane_bez_brakow["Age"] = dane_bez_brakow["Age"].astype('int')
dane_bez_brakow["Embarked"] = dane_bez_brakow["Embarked"].astype('string')

In [16]:
for index, row in braki.iterrows():
    if row["Procent braków"] > 0.0:
        try:
            if data[do_uzupelnienia[index]].dtype == 'object' or data[index].dtype == 'object':
                ### Dane bez możliwości obliczenia średniej, użyta więc mediana
                ### Stworzenie słownika gdziue kluczem jest wartosc kolumny grupujacej,
                ### a wartoscia mediana z uzupelnianej wartosci.
                unikalne = dane_bez_brakow.Sex.unique()
                mediany = {}
                for wartosc in unikalne:
                    lista = list(dane_bez_brakow[dane_bez_brakow.Sex == wartosc]["Embarked"])
                    lista.sort()
                    mediany[wartosc] = (lista[int(len(lista) // 2)])
                for i, r in data.isna().iterrows():
                    if r[index] == True:
                        data.at[i, index] = mediany[data.loc[i][do_uzupelnienia[index]]]
            else:
                ### Dane umożliwiają obliczenie średniej
                srednie = dane_bez_brakow.groupby(do_uzupelnienia[index])[index].mean()
                for i, r in data.isna().iterrows():
                    if r[index] == True:
                        data.at[i, index] = str(int(srednie[data.loc[i][do_uzupelnienia[index]]]))
        except KeyError:
            print("Atrybut nie został uwzględniony w słowniku")


Ponowne obliczenie wartości brakujących w celu sprawdzenia czy poprawnie zostały uzupełnione

In [17]:
braki = pandas.DataFrame(data.isna().sum())
braki.columns = ["Ilość braków"]

procenty = [x / len(data) for x in data.isna().sum()]
procnety = np.array(procenty)
procenty = np.around(procenty, 3)

braki["Procent braków"] = procenty * 100

braki

,Ilość braków,Procent braków
Survived,0,0.0
Pclass,0,0.0
Name,0,0.0
Sex,0,0.0
Age,0,0.0
SibSp,0,0.0
Parch,0,0.0
Ticket,0,0.0
Fare,0,0.0
Embarked,0,0.0


## Podsumowanie

Wykonane czynnosci:
1. Usunięcie wierszy zduplikowanych w ilosci 3
2. Naprawa błędnych wartośći w kolumnie 'Age' tj. zamiana wszystkich liczb zmienno przecinkowych na liczby naturalne
3. Naprawa błednych wartości w kolumnie 'Sex' tj. zamiana błednych wartosci z literówkami na ich prawidłowe wartosci
4. Obliczenie ilości brakujących danych w poszczególnych atrybutach na podstawie, której podejmowana były kolejne decyzje.
5. Usuniecie całkowicie atrybutu 'Cabin' ze względu na 77% brak danych.
6. Automatyczne uzupełnienie reszty danych w przypadku danych ilościowych zastepowane zostały srednia wartoscia dla danej grupy, natomiast w przypadku danych jakościowych była to mediana. Automatyczność tego procesu polega na tym iż uzytkwonik okresla słownik gdzie kluczem jest atrybut który ma zostac uzupełniony, a wartościa atrybut po którym bedą grupowane dany, reszte program wykonuje automatycznie przy pomocy algorytmu oraz wbudowanych funkcji

Wyczyszczone dane:

In [20]:
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,ship
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",S,Titanic
...,...,...,...,...,...,...,...,...,...,...,...
888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30,S,Titanic
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,37,1,2,W./C. 6607,"23,45",S,Titanic
890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30,C,Titanic


Zapis danych do pliku 'TitanicCleaned'

In [21]:
data.to_csv("TitanicCleaned.csv")